In [26]:
import torch 
from torch import Tensor
from model_helpers import preprocess_text
import pickle
from rnn_model import EncoderRNN, DecoderRNN
import random
device = "cpu"


# Load dataset
dataset_path = "./model_trained/stored_dataset.pickle"
with open(dataset_path, 'rb') as f :
    dataset = pickle.load(f)

    
# Load the encoder and decoder models
encoder_path = "./model_trained/encoder.pt"
decoder_path = "./model_trained/decoder.pt"
input_size = 100
hidden_size = 256
output_size = len(dataset.chords_set)

encoder = EncoderRNN(input_size, hidden_size, device)
decoder = DecoderRNN(hidden_size, output_size, device)
encoder.load_state_dict(torch.load(encoder_path))
decoder.load_state_dict(torch.load(decoder_path))

<All keys matched successfully>

In [27]:
# Method to evaluate the model 

def predict(encoder, decoder, sentence, max_length=7039):
    with torch.no_grad():
        input_tensor = Tensor([dataset.vectorize(w) for w in preprocess_text( sentence)]).to(device)
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.initHidden()

        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)


        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(
                input_tensor[ei], encoder_hidden)
            encoder_outputs[ei] = encoder_output[0, 0]
        decoder_input = torch.tensor([dataset.chord2id[dataset.draw_chord()[0]]], device=device)

        decoder_hidden = encoder_hidden

        decoded_words = [random.sample(dataset.chords_set,1)[0]]
        # decoder_attentions = torch.zeros(max_length, max_length)

        for di in range(1,4):
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden)
            topv, topi = decoder_output.data.topk(1)
            decoded_words.append(dataset.id2chord[topi.item()])

            decoder_input = topi.squeeze().detach()

        return decoded_words

In [37]:
# Input your lyrics and Tadaaaaaa
lyrics = '''
Today is gonna be the day that they're gonna throw it back to you
And by now, you should've somehow realised what you gotta do
'''

predict(encoder, decoder,lyrics )

['Fm6', 'D#', 'A#', 'D#']